### 데이터 값 찾기

* 항목과 값 찾기: 확정급여채무의 현재가치, 사외적립자산의 공정가치

In [1]:
import pandas as pd
# 사업보고서 제무제표 주석 문서 URL 목록
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/fs_notes_urls/fs_notes_url_all.csv"
fs_notes_all = pd.read_csv(file_path, dtype=object)
# 회사코드 정렬
fs_notes_all = fs_notes_all.sort_values(by=['corp_code'], ascending=True)

In [2]:
import traceback
from time import sleep
from tqdm import tqdm

import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import myutil, mydart

corp_codes = ['00103662']
rpt_number = '20230321000645'

# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/items_from_section/"

for corp_code in tqdm(corp_codes):
    str_expr = "corp_code == @corp_code"
    df_doc = fs_notes_all.query(str_expr)
    doc_count = df_doc.shape[0]
    data_list = []
    for i, doc in df_doc.iterrows():        
        if doc['rpt_num'] == rpt_number:
            sleep(1.0)
            try:
                val_a = val_b = 0.0                
                section_title = None
                unit = None
                doc_result = doc['doc_result']
                if doc_result == 'No Financial Statement Notes':
                    data_list.append({'corp_code' : doc['corp_code'], 'stock_code' : doc['stock_code'], 'corp_cls' : doc['corp_cls'], 'corp_name' : doc['corp_name'], 
                                    'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 
                                    'doc_title' : doc['doc_title'], 'doc_result' : doc_result, 'section_title' : None, 
                                    'unit' : None, 'val_a' : None, 'val_b' : None, 'item_result' : None, 'doc_url' : None})
                else:
                    doc_url = doc['doc_url']                    
                    soup = mydart.get_document(doc_url)
                    subtitle_list = mydart.get_subtitles(soup)
                    section_title = mydart.find_section_title(subtitle_list)
                    if section_title is None:       # 해당 Section을 찾지 못했을 경우
                        data_list.append({'corp_code' : doc['corp_code'], 'stock_code' : doc['stock_code'], 'corp_cls' : doc['corp_cls'], 'corp_name' : doc['corp_name'], 
                                        'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 
                                        'doc_title' : doc['doc_title'], 'doc_result' : doc_result, 'section_title' : None, 
                                        'unit' : None, 'val_a' : None, 'val_b' : None, 'item_result' : None, 'doc_url' : doc_url})                        
                    else:                                         
                        section_page = mydart.get_section_html(soup, subtitle_list, section_title)
                        # 해당 Section에서 사용된 금액 단위
                        unit = mydart.find_unit_from_section(section_page)
                        # 당기말의 "확정급여채무"와 "사외적립자산" 금액 찾기
                        val_a, val_b, item_result = mydart.find_item_from_section(section_page)      
                        data_list.append({'corp_code' : doc['corp_code'], 'stock_code' : doc['stock_code'], 'corp_cls' : doc['corp_cls'], 'corp_name' : doc['corp_name'], 
                                        'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 
                                        'doc_title' : doc['doc_title'], 'doc_result' : doc_result, 'section_title' : section_title, 
                                        'unit' : unit, 'val_a' : val_a, 'val_b' : val_b, 'item_result' : item_result, 'doc_url' : doc_url})
            except ValueError as err:
                print(doc['corp_code'], doc['corp_name'], doc['rpt_num'], doc['rpt_name'], doc['rpt_url'], doc['doc_title'], doc['doc_url'])
                print(err)
                data_list.append({'corp_code' : doc['corp_code'], 'stock_code' : doc['stock_code'], 'corp_cls' : doc['corp_cls'], 'corp_name' : doc['corp_name'], 
                                'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 
                                'doc_title' : doc['doc_title'], 'doc_result' : doc_result, 'section_title' : section_title, 
                                'unit' : unit, 'val_a' : val_a, 'val_b' : val_b, 'item_result' : 'ValueError', 'doc_url' : doc_url})                   
                break
            except Exception as e:
                print(doc['corp_code'], doc['corp_name'], doc['rpt_num'], doc['rpt_name'], doc['rpt_url'], doc['doc_title'], doc['doc_url'])
                err_msg = traceback.format_exc()
                print(err_msg)
                data_list.append({'corp_code' : doc['corp_code'], 'stock_code' : doc['stock_code'], 'corp_cls' : doc['corp_cls'], 'corp_name' : doc['corp_name'], 
                                'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 
                                'doc_title' : doc['doc_title'], 'doc_result' : doc_result, 'section_title' : section_title, 
                                'unit' : unit, 'val_a' : val_a, 'val_b' : val_b, 'item_result' : 'Exception', 'doc_url' : doc_url})                     
                break

100%|██████████| 1/1 [00:02<00:00,  2.44s/it]


In [3]:
data_list

[{'corp_code': '00103662',
  'stock_code': '017040',
  'corp_cls': 'Y',
  'corp_name': '광명전기',
  'rpt_num': '20230321000645',
  'rpt_name': '사업보고서 (2022.12)',
  'doc_title': '5. 재무제표 주석',
  'doc_result': 'OK',
  'section_title': '21. 퇴직급여제도',
  'unit': '원',
  'val_a': 7476414932.0,
  'val_b': 4876440352.0,
  'item_result': '확정급여채무(기말), 사외적립자산',
  'doc_url': 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20230321000645&dcmNo=9080328&eleId=22&offset=886611&length=425485&dtd=dart3.xsd'}]

In [31]:
subtitle_list

['1. 일반사항',
 '2. 재무제표 작성기준',
 '3. 중요한 회계정책',
 '4. 제ㆍ개정된 기준서의 적용',
 '5. 중요한 회계적 판단, 추정 및 가정',
 '6. 공표되었으나 아직 시행되지 않은 회계기준',
 '7. 영업부문',
 '8. 단기금융상품 및 장기금융상품',
 '9. 매출채권및기타채권',
 '10. 재고자산',
 '11. 기타자산',
 '12. 관계기업투자',
 '13. 유형자산',
 '14. 무형자산',
 '15. 매입채무및기타채무',
 '16. 기타유동부채',
 '17. 차입금',
 '18. 종업원급여',
 '19. 자본금 및 자본잉여금',
 '20. 이익잉여금',
 '21. 기타자본',
 '22. 주식기준보상',
 '23. 매출액 및 매출원가',
 '24. 판매비와관리비',
 '25. 비용의 성격별 분류',
 '26. 기타손익',
 '27. 금융손익',
 '28. 법인세비용',
 '29. 주당손익',
 '30. 우발사항 및 약정사항',
 '31. 특수관계자와의 거래',
 '32. 현금흐름표',
 '33. 금융상품',
 '34. 재무위험관리']

In [32]:
section_title

'18. 종업원급여'

In [33]:
section_page

'18. 종업원급여\n   <br/>\n   <br/>\n   18.1 당기말 및 전기말 현재 퇴직급여부채의 내역은 다음과 같습니다.\n  </p>\n  <table class="nb" width="625">\n   <colgroup>\n    <col width="300"/>\n    <col width="325"/>\n   </colgroup>\n   <tbody>\n    <tr>\n     <td height="22" valign="TOP" width="300">\n      <br/>\n     </td>\n     <td align="RIGHT" height="22" valign="TOP" width="325">\n      (단위: 천원)\n     </td>\n    </tr>\n   </tbody>\n  </table>\n  <table border="1" width="623">\n   <colgroup>\n    <col width="208"/>\n    <col width="208"/>\n    <col width="207"/>\n   </colgroup>\n   <thead>\n    <tr>\n     <th align="CENTER" height="20" width="208">\n      구분\n     </th>\n     <th align="CENTER" height="20" width="208">\n      당기말\n     </th>\n     <th align="CENTER" height="20" width="207">\n      전기말\n     </th>\n    </tr>\n   </thead>\n   <tbody>\n    <tr>\n     <td height="20" valign="TOP" width="208">\n      확정급여채무의 현재가치\n     </td>\n     <td align="RIGHT" height="20" valign="TOP" width="208">\n      8,315,410\n

In [34]:
idx = 0
idx = subtitle_list.index(section_title)
next_title = subtitle_list[idx + 1]
# 문서에서 해당 Section 시작위치와 종료위치 찾기
doc_html = soup.prettify()
p0 = doc_html.find(section_title)
p1 = doc_html.find(next_title)
section_html = doc_html[p0:p1]     # 문서 Slicing

In [22]:
next_title

'19. 자본금 및 자본잉여금'

In [23]:
p0, p1

(161196, 171673)

In [24]:
doc_html

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<html style="border:0">\n <head>\n  <title>\n  </title>\n  <meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible"/>\n  <meta content="gdi" http-equiv="X-UA-TextLayoutMetrics"/>\n  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n  <link href="/css/report_xml.css" rel="stylesheet" type="text/css"/>\n </head>\n <body bgcolor="#FFFFFF">\n  <p>\n   <br/>\n  </p>\n  <p class="section-2">\n   <a name="toc1">\n    5. 재무제표 주석\n   </a>\n  </p>\n  <p style=\'font-family:"굴림", Gulim;font-size:11pt;\'>\n   <br/>\n   1. 일반사항\n  </p>\n  <p style=\'font-family:"굴림", Gulim;font-size:11pt;\'>\n   <br/>\n   일진디스플레이 주식회사(이하 "당사")는 1988년 4월 일진다이아몬드공업사로 설립되어 현재는 사파이어 웨이퍼 및 터치스크린 패널의 제조를 주된 목적사업으로 하고 있으며, 2002년 1월 9일 자로 한국거래소가 개설한 유가증권시장에 주식을 상장하였습니다. 당사는 2004년 12월 1일을 기준일로 하여 공업용 합성다이아몬드 사업부문 (현재의 일진다이아몬드 주식회사)을 인적분할의 방법으로 설립하였으며, 존속회사인 당사의 상호를 일진다이아몬드 주식회사에서 일진디스플레이 주식회사로 변경하였습

In [35]:
unit = mydart.find_unit_from_section(section_page)

In [36]:
unit

'천원'

In [25]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html style="border:0">
<head>
<title></title>
<meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible"/>
<meta content="gdi" http-equiv="X-UA-TextLayoutMetrics"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="/css/report_xml.css" rel="stylesheet" type="text/css"/>
</head>
<body bgcolor="#FFFFFF">
<p><br/></p>
<p class="section-2"><a name="toc1">5. 재무제표 주석</a></p>
<p style='font-family:"굴림", Gulim;font-size:11pt;'><br/>1. 일반사항</p>
<p style='font-family:"굴림", Gulim;font-size:11pt;'><br/>일진디스플레이 주식회사(이하 "당사")는 1988년 4월 일진다이아몬드공업사로 설립되어 현재는 사파이어 웨이퍼 및 터치스크린 패널의 제조를 주된 목적사업으로 하고 있으며, 2002년 1월 9일 자로 한국거래소가 개설한 유가증권시장에 주식을 상장하였습니다. 당사는 2004년 12월 1일을 기준일로 하여 공업용 합성다이아몬드 사업부문 (현재의 일진다이아몬드 주식회사)을 인적분할의 방법으로 설립하였으며, 존속회사인 당사의 상호를 일진다이아몬드 주식회사에서 일진디스플레이 주식회사로 변경하였습니다. <br/><br/>당사의 본사는 충청북도 음성군에, 공장은 충청북도 음성군과 경기도 평택시에 위치하고 있습니다.</p>
<p style='font-family:"굴

In [52]:
import re
tags = soup.find_all(string = True)
subtitle_list = []
ex_subject1 = re.compile('\d-+\d+[.]+[^0-9]+')      # 숫자-숫자+'.'+숫자가아닌모든문자열
ex_subject2 = re.compile('\d-+\d-+\d+[.]+[^0-9]+')  # 숫자-숫자-숫자+'.'+숫자가아닌모든문자열
for t in tags:
    if ex_subject1.match(t.text) or ex_subject2.match(t.text):
        pass
    else:
        subjects1 = re.findall('\d+[.]+\s+[^0-9]+', t.text)  # 숫자+'.'+공백+숫자가아닌모든문자열     
        subjects2 = re.findall('\d+[.]+[^0-9]+', t.text)    # 숫자+'.'+숫자가아닌모든문자열
        subjects_all = subjects1 + subjects2
        subjects = list(set(subjects_all))      # 중복 제거
        if len(subjects) > 0:
            for s in subjects:
                subtitle_list.append(s.rstrip())    # 문자열 끝의 공백 없애고 list에 추가
                
ordered_subtitle_list = []
prev_no = 0             
for s in subtitle_list:
    this_no = int(s.split(".")[0])
    if this_no == prev_no + 1 or this_no == prev_no + 2 or this_no == prev_no + 3:
        ordered_subtitle_list.append(s)
        prev_no = this_no                

In [53]:
ordered_subtitle_list

['1. 일반사항',
 '2. 재무제표 작성기준 및 중요한 회계정책',
 '3. 중요한 회계정책',
 '4. 위험관리 및 자본관리에 관한 기타 공시',
 '5. 공정가치',
 '6. 범주별 금융상품',
 '7. 현금및현금성자산',
 '8. 매출채권및기타채권',
 '9. 기타자산',
 '10. 재고자산',
 '11. 유형자산',
 '12. 매입채무및기타채무',
 '13. 단기차입금',
 '14. 기타유동부채',
 '15. 충당부채',
 '16. 확정급여부채',
 '17. 자본금',
 '18. 자본잉여금',
 '19. 이익잉여금',
 '20. 매출 및 매출원가',
 '21. 판매비와관리비',
 '22. 비용의 성격별 분류',
 '23. 기타수익 및 기타비용',
 '24. 법인세비용과 이연법인세자산(부채)',
 '25. 주당이익',
 '26. 특수관계자',
 '27. 현금흐름표',
 '28. 보험가입자산',
 '29. 담보제공자산',
 '30. 제공받은 담보 및 지급보증',
 '31. 우발채무 및 소송사건 등']

In [21]:
subjects2

[]